In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import PredefinedSplit
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy import stats

In [2]:
def R2OOS(y_true, y_forecast, expanding_mean):
    SSres = sum((y_true - y_forecast)**2)
    SStot = sum((y_true - expanding_mean)**2)
    fraction = SSres / SStot
    R2OOS = 1 - fraction
    return 100 * R2OOS  #for %

In [3]:
def cw(y_forecast, y_true, expanding_mean):
    mspe_model = np.mean((y_true - y_forecast) **2)
    mspe_benchmark = np.mean((y_true - expanding_mean)**2)
    numerator = mspe_model - mspe_benchmark

    var_mspe_model = np.mean(((y_true - y_forecast) ** 2 - mspe_model) ** 2)
    var_mspe_benchmark = np.mean(((y_true - expanding_mean) ** 2 - mspe_benchmark) ** 2)
    #denominator = np.sqrt(np.mean(var_mspe_model + var_mspe_benchmark))
    denominator = np.sqrt(var_mspe_model + var_mspe_benchmark) 

    cw_statistic = numerator / denominator
    
    #SE_cw_statistic = np.sqrt(np.mean(var_mspe_model + var_mspe_benchmark))
    SE_cw_statistic = np.sqrt(var_mspe_model + var_mspe_benchmark)  

    n = len(y_true)
    df = n-1
    t_stat = cw_statistic / SE_cw_statistic
    p_value = 2* (1 - stats.t.cdf(np.abs(t_stat),df))

    return 1-p_value

In [4]:
def run_elastic_net(X_train, Y_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test.reshape(1, -1))
    N_train = int(np.round(np.size(X_train_scaled,axis=0)*0.85))
    N_val = np.size(X_train_scaled,axis=0)-N_train
    test_fold =  np.concatenate(((np.full((N_train),-1),np.full((N_val),0))))
    ps = PredefinedSplit(test_fold.tolist())
    model = ElasticNetCV(cv=ps,
                         l1_ratio = [0.1, 0.3, 0.5, 0.7, 0.9, 1],
                         n_jobs =-1)
    Y_pred = np.full((1, Y_train.shape[1]), np.nan)  

    for i in range(Y_train.shape[1]):  
        model.fit(X_train_scaled, Y_train[:, i])
        Y_pred[0, i] = model.predict(X_test_scaled)[0]  

    return Y_pred

In [5]:
# === SETTINGS ===
EXCESS_RETURN_PATH = '/Users/Siebe.van.Wingerden/Desktop/Python Codes/xr2.xlsx'
FWD_RATE_PATH = '/Users/Siebe.van.Wingerden/Desktop/Python Codes/fwd2.xlsx'
OOS_START_DATE = "1990-01-01"
OOS_END_DATE = "2018-12-01"
FULL_START_DATE = "1971-08-01"  # for full sample filtering

# === MERGE DATASETS ===
df_xr = pd.read_excel(EXCESS_RETURN_PATH)
df_fwd = pd.read_excel(FWD_RATE_PATH)
df = pd.merge(df_xr, df_fwd, on="Date", suffixes=("_xr", "_fwd"))
df.set_index("Date", inplace=True) #set Date as index

# === FILTER DATA ===
regressor_maturities = ["12 m", "24 m", "36 m", "48 m", "60 m", "72 m", "84 m", "96 m", "108 m", "120 m"]
fwd_cols = [m + "_fwd" for m in regressor_maturities]
target_maturities = ["24 m", "36 m", "48 m", "60 m", "84 m", "120 m"]
target_cols = [m + "_xr" for m in target_maturities]
df = df[(df.index >= FULL_START_DATE) & (df.index <= OOS_END_DATE)].copy() #only include the dates in range of Bianche et al. 

# === Prepare arrays ===
Y = df[target_cols].to_numpy()
X_fwd = df[fwd_cols].to_numpy()
dates = df.index.to_numpy()
n_obs = Y.shape[0]
n_maturities = Y.shape[1]
start_oos_idx = df.index.get_loc(pd.to_datetime(OOS_START_DATE))

# === Run Expanding Window Forecast from 1990-01 ===
expanding_mean_check = np.full_like(Y, np.nan, dtype=np.float64)
expanding_mean_check[0] = 0
for t in range(1,n_obs):
    expanding_mean_check[t, :] = Y[:t, :].mean(axis=0)

expanding_mean_df = pd.DataFrame(expanding_mean_check, columns=target_maturities, index=df.index)
expanding_mean_df = expanding_mean_df.iloc[start_oos_idx:] #delete in sample period

# === Run OOS Forecast ===
print("Running Elastic Net with expanding window...\n")
y_preds = np.full_like(Y, np.nan)

for t in range(start_oos_idx + 11, len(Y)):
    y_preds[t, :] = run_elastic_net(X_fwd[:t, :], Y[:t, :], X_fwd[t, :])
    print(f"Forecast {t+1 - start_oos_idx}/{len(Y) - start_oos_idx} ({df.index[t].strftime('%Y-%m')})")

# === Performance Evaluation #1===
print("\n=== Elastic Net Regression OOS Performance ===")
for i, col in enumerate(target_maturities):
    y_true = Y[start_oos_idx + 11:, i]
    y_forecast = y_preds[start_oos_idx + 11:, i]
    benchmark = expanding_mean_df[col].to_numpy()[11:]
    r2 = R2OOS(y_true, y_forecast, benchmark)
    mspe = np.mean((y_true - y_forecast) ** 2)
    cw_p = cw(y_forecast, y_true, benchmark)
  
    print(f"{col}: R²OOS={r2:.4f}%, MSPE={mspe:.4f}, p-value={cw_p:.4f}")

Running Elastic Net with expanding window...

Forecast 12/348 (1990-12)
Forecast 13/348 (1991-01)
Forecast 14/348 (1991-02)
Forecast 15/348 (1991-03)
Forecast 16/348 (1991-04)
Forecast 17/348 (1991-05)
Forecast 18/348 (1991-06)
Forecast 19/348 (1991-07)
Forecast 20/348 (1991-08)
Forecast 21/348 (1991-09)
Forecast 22/348 (1991-10)
Forecast 23/348 (1991-11)
Forecast 24/348 (1991-12)
Forecast 25/348 (1992-01)
Forecast 26/348 (1992-02)
Forecast 27/348 (1992-03)
Forecast 28/348 (1992-04)
Forecast 29/348 (1992-05)
Forecast 30/348 (1992-06)
Forecast 31/348 (1992-07)
Forecast 32/348 (1992-08)
Forecast 33/348 (1992-09)
Forecast 34/348 (1992-10)
Forecast 35/348 (1992-11)
Forecast 36/348 (1992-12)
Forecast 37/348 (1993-01)
Forecast 38/348 (1993-02)
Forecast 39/348 (1993-03)
Forecast 40/348 (1993-04)
Forecast 41/348 (1993-05)
Forecast 42/348 (1993-06)
Forecast 43/348 (1993-07)
Forecast 44/348 (1993-08)
Forecast 45/348 (1993-09)
Forecast 46/348 (1993-10)
Forecast 47/348 (1993-11)
Forecast 48/348 (1

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.505e+03, tolerance: 6.186e+00
  model = cd_fast.enet_coordinate_descent(


Forecast 345/348 (2018-09)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.790e+00, tolerance: 3.222e+00
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.784e+03, tolerance: 6.198e+00
  model = cd_fast.enet_coordinate_descent(


Forecast 346/348 (2018-10)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.261e+02, tolerance: 3.227e+00
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.821e+03, tolerance: 6.211e+00
  model = cd_fast.enet_coordinate_descent(


Forecast 347/348 (2018-11)
Forecast 348/348 (2018-12)

=== Elastic Net Regression OOS Performance ===
24 m: R²OOS=-11.0897%, MSPE=1.6830, p-value=0.0146
36 m: R²OOS=-8.1480%, MSPE=6.2688, p-value=0.0029
48 m: R²OOS=-4.5630%, MSPE=12.2208, p-value=0.0009
60 m: R²OOS=-2.5350%, MSPE=19.3547, p-value=0.0003
84 m: R²OOS=-2.2739%, MSPE=36.9301, p-value=0.0002
120 m: R²OOS=3.9721%, MSPE=65.8690, p-value=0.0002


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.847e+03, tolerance: 3.230e+00
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.870e+03, tolerance: 6.220e+00
  model = cd_fast.enet_coordinate_descent(
